1. Clarifier(prompt) -> requirements
2. Namer(requirements) -> name
3. Planner(requirements) -> execution plan
4. for each step of the plan:
       Executor(step) -> code 

In [1]:
!uv add toyaikit==0.0.8

Resolved 198 packages in 0.75ms
Audited 194 packages in 102ms


In [2]:
from openai import OpenAI

from toyaikit.tools import Tools
from toyaikit.llm import OpenAIClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner, DisplayingRunnerCallback

from pydantic import BaseModel
from typing import Literal

In [3]:
import os
import shutil

def start(project_name):
    if not project_name:
        print("Project name cannot be empty.")
        return False

    if os.path.exists(project_name):
        print(f"Directory '{project_name}' already exists. Please choose a different name or remove the existing directory.")
        return False

    shutil.copytree('django_template', project_name)
    print(f"Django template copied to '{project_name}' directory.")

    return True

## Clarifier

In [12]:
clarifier_instructions = """
You are the Requirement Clarifier for a multi-agent AI coding assistant project.
Your job is to come up with EXACTLY three multiple-choice questions to clarify the
user's goals and constraints. 

Ask these questions one by one until you get the answers to all. Output "READY" when
all questions are answered

Don't focus on implementation details (like target platform, etc)
Only on the functional requirements.

Examples of not relevant questions:

- target platform (web, mobile, text)
- data persistance

The final implementation will be a web-based application implemented as a Django service,
so don't ask questions about interface.
""".strip()

In [13]:
chat_interface = IPythonChatInterface() 

clarifier = OpenAIResponsesRunner(
    developer_prompt=clarifier_instructions,
    chat_interface=chat_interface,
    llm_client=OpenAIClient(model='gpt-4o-mini')
)

In [15]:
def stop_on_ready(messages):
    if len(messages) == 0:
        return False

    entry = messages[-1]
    if entry.type == 'message':
        text = entry.content[0].text
        if 'READY' in text:
            return True

    return False

In [14]:
clarifier_result = clarifier.run(stop_criteria=stop_on_ready)

You: todo list


-> Response received


You: e


-> Response received


You: d


-> Response received


You: ab


-> Response received


You: stop


Chat ended.


In [35]:
total_cost = clarifier_result.cost

In [18]:
messages = clarifier_result.new_messages

In [20]:
qa_prompt_builder = []

for m in messages:
    if not (hasattr(m, 'content') and hasattr(m, 'role')):
        continue

    if m.role == 'user':
        qa_prompt_builder.append('user: ')
        qa_prompt_builder.append(m.content)
        qa_prompt_builder.append('\n')

    if m.role == 'assistant':
        qa_prompt_builder.append('assistant: ')
        for p in m.content:
            if p.type != 'output_text':
                continue
            qa_prompt_builder.append(p.text)
            qa_prompt_builder.append('\n')

    qa_prompt_builder.append('\n')

qa_prompt = ''.join(qa_prompt_builder).strip()
print(qa_prompt)

user: todo list

assistant: 1. What specific features do you want in your to-do list application?  
   a) Basic task creation and management  
   b) Task prioritization  
   c) Deadline reminders  
   d) Task sharing with others  
   e) All of the above  
   f) Other (please specify)

user: e

assistant: 2. How do you want users to interact with the tasks in the application?  
   a) Creating and editing tasks only  
   b) Adding comments or notes to tasks  
   c) Marking tasks as complete or incomplete  
   d) All of the above  
   e) Other (please specify)

user: d

assistant: 3. What type of user roles do you want to implement in the application?  
   a) Individual users only  
   b) Team members with different access levels  
   c) Admin users for managing the application  
   d) No specific roles needed  
   e) Other (please specify)

user: ab

assistant: READY


## Namer

In [21]:
namer_instructions = """
your task is to come up with a name for a project
name the project with a unique, meaningful title that reflects its purpose and avoids naming conflicts

generate a slug (a URL- and filesystem-safe version of the project name)
to serve as the project folder name.
the slug should be short, up to 15 characters
""".strip()

class ProjectName(BaseModel):
    name: str
    slug: str

In [22]:
namer = OpenAIResponsesRunner(
    developer_prompt=namer_instructions,
    llm_client=OpenAIClient(model='gpt-4o-mini')
)

In [23]:
namer_result = namer.loop(
    prompt=qa_prompt,
    output_format=ProjectName
)

In [25]:
name = namer_result.last_message

In [26]:
name.slug

'taskmaster'

In [37]:
total_cost = total_cost + namer_result.cost
total_cost

CostInfo(input_cost=Decimal('0.0001968'), output_cost=Decimal('0.0001218'), total_cost=Decimal('0.0003186'))

In [27]:
start(name.slug)

Django template copied to 'taskmaster' directory.


True

## Planner

In [30]:
from tools import AgentTools
from pathlib import Path

In [31]:
agent_tools = AgentTools(Path(name.slug))

In [33]:
read_only_tools = Tools()
read_only_tools.add_tool(agent_tools.read_file)
read_only_tools.add_tool(agent_tools.search_in_files)
read_only_tools.add_tool(agent_tools.see_file_tree)

In [38]:
planner_instructions = """
You are a planning agent responsible for designing the application based on functional requirements.
You assist the coding agent by generating a detailed, actionable implementation plan.

Your Role:

- You get a set of functional requirements from the clarifier agent.
- You do not modify the codebase directly, but you must describe precisely the changes
  and actions that should be taken.
- Your goal is to translate the user requirements into a clear step-by-step plan for
  how to implement the desired functionality in the existing Django project.

Instructions:

- Check the file structure to better plan your work
- Always include styling in the plan
- Data processing should happen in backend, not templates
- Make sure the main interaction elements are accessible from the home page
- Focus on clarity, modularity, and maintainability in your plan
- Include tests
- Don't include the exact code in the output, focus on the instructions
- Don't overcomplicate. The output should be an MVP, not a production website

Your output will be consumed by a coding agent, so it must be precise, unambiguous,
and broken down into logical steps.

Only include coding instructions. The output will not be read by humans - only 
by the coding agent. The coding agent can only follow your suggested actions, but 
cannot plan, think, or update the requirements.

The coding agent will see only one step at a time, so make steps self-contained. 
Don't add "notes" as steps, steps can only include coding instructions.
""".strip()

In [39]:
class PlanStep(BaseModel):
    name: str
    detailed_description: str

class Plan(BaseModel):
    overview: str
    steps: list[PlanStep]

In [40]:
planner = OpenAIResponsesRunner(
    developer_prompt=planner_instructions,
    tools=read_only_tools,
    llm_client=OpenAIClient(model='gpt-5')
)

In [41]:
callback = DisplayingRunnerCallback(chat_interface)

In [42]:
planning_result = planner.loop(
    prompt=qa_prompt,
    callback=callback,
    output_format=Plan,
)

-> Response received


-> Response received


-> Response received


-> Response received


-> Response received


-> Response received


-> Response received


-> Response received


-> Response received


In [45]:
plan = planning_result.last_message

In [46]:
plan.overview

'Implement a multi-user to-do list with task creation/management, prioritization, deadline reminders, task sharing, comments, completion toggling, and team roles. Ensure main interactions are accessible on the home page, with backend-driven processing and minimal yet clear styling using existing Tailwind + styles.css. Provide URLs, views, forms, models, templates, settings, management command, and tests. Keep modular and maintainable.'

In [49]:
for step in plan.steps:
    print(step.name)
    print(step.detailed_description)
    print()

Create data models and initial migrations
1) In myapp/models.py, define the following models:
- Team: fields name (str), owner (FK to User), created_at (datetime). Unique constraint on (owner, name). __str__ returns name.
- TeamMembership: fields team (FK Team, related_name="memberships"), user (FK User), role (choices: OWNER, MANAGER, MEMBER, VIEWER), created_at. Unique constraint on (team, user).
- Task: fields title (str, required), description (text, optional), owner (FK User), team (FK Team, null/blank allowed), priority (choices: LOW, MEDIUM, HIGH; default MEDIUM), due_date (date/time, optional), is_completed (bool, default False), completed_at (datetime, nullable), created_at, updated_at. ManyToMany to User as shared_with (blank allowed). Add model methods: can_view(user), can_edit(user) to enforce: owner true; shared_with true; team members true for view; edit allowed for owner, shared_with, and team roles OWNER/MANAGER/MEMBER (VIEWER view-only). Save() should set completed_at 

In [53]:
extra_steps = [
    PlanStep(
        name="Make sure the project is runnable locally",
        detailed_description="Run diagnostics to make it possible to run it with runserver"
    ),
    PlanStep(
        name="Tests",
        detailed_description="Make sure the code is covered with tests"
    ),
]

In [54]:
steps = plan.steps + extra_steps

In [57]:
planning_result.cost

CostInfo(input_cost=Decimal('0.03232125'), output_cost=Decimal('0.06183'), total_cost=Decimal('0.09415125'))

In [58]:
total_cost = total_cost + planning_result.cost
total_cost.total_cost

Decimal('0.09446985')

## Executor

In [55]:
executor_instructions = """
You are a coding agent responsible for applying specific modifications to
a Django project based on instructions from a planning agent.

You are given a plan and you need to execute it step by step. 
Follow the instructions precisely and modify the codebase directly
using the tools available to you.

Each time you execute a step, give a short explanation of what you did and which
step number you finished.

You do not ask questions. You do not suggest. You execute.

All changes must adhere to Django best practices and fit cleanly into the
existing project structure.

## Project Context

This is a Django 5.2.4 web application scaffolded with common conventions and clean separation of
concerns.

Key technologies and constraints:

- Django 5.2.4
- SQLite for local database
- TailwindCSS for all styling
- HTML templates used for rendering

## File Notes

- `manage.py`: Entry point for Django commands
- `myapp/`: Main application logic, templates, and data models
- `myproject/`: Project-wide configuration and routing
- `templates/`: Shared layout templates (e.g., base.html)
- `static/css/styles.css`: Custom styling, especially important for forms

## Execution rules

- Do not run the development server (e.g. runproject), but you may run diagnostic or validation commands.
- Use TailwindCSS to create clean and user-friendly UI components. Avoid creating custom styles unless really necessary.
- Include emojis and pictograms where it enhances the user interface. Font Awesome is available.
- Do not place logic in templates. Use views or models for data processing and transformation.
- You can create, edit, or delete any files as needed to complete your task.

After execution is complete, show a short summary.

Act with precision. Implement the plan.
"""

In [60]:
all_tools = Tools()
all_tools.add_tools(agent_tools)

executor = OpenAIResponsesRunner(
    tools=all_tools,
    developer_prompt=executor_instructions,
    llm_client=OpenAIClient(model='gpt-5-mini'),
)

In [67]:
prompt_template = """
Project overview: <OVERVIEW>{overview}</OVERVIEW>

Current step - step #{step_number}:

<STEP>
{step_name}

{step_description}
</STEP>

Previous steps (already completed by you):

<PREVIOUS_STEPS>
{previous_steps}
</PREVIOUS_STEPS>

Files created or changed by you so far: 

<CHANGED_FILES>
{files}
</CHANGED_FILES>

File tree: 

<FILES>
{file_tree}
<FILES>
""".strip()

In [68]:
import json

def find_updated_files(messages):
    updated_files = set()

    for m in messages:
        if not hasattr(m, 'type'):
            continue 
        if not m.type == 'function_call':
            continue
        if m.name != 'write_file':
            continue
    
        try:
            args = json.loads(m.arguments)
            updated_file = args['filepath']
            updated_files.add(updated_file)
        except json.JSONDecodeError as e: 
            print(e, 'failed to parse')

    return updated_files

In [74]:
class ExecutionReport(BaseModel):
    summary: str
    complete: bool

In [70]:
all_changed_files = set()

In [ ]:
for i in range(len(steps)):
    if i < 8:
        continue

    previous_steps = steps[:i]
    current_step = steps[i]
    print(f'implementing step #{i + 1}: {current_step.name}...')
    print(current_step.detailed_description)

    previous_steps_text = '\n'.join([f'- {s.name}' for s in previous_steps])
    file_tree = '\n'.join(agent_tools.see_file_tree())
    
    prompt = prompt_template.format(
        overview=plan.overview,
        step_number=i,
        step_name=current_step.name,
        step_description=current_step.detailed_description,
        previous_steps=previous_steps_text,
        files='\n'.join(all_changed_files),
        file_tree=file_tree
    )

    print('prompt:')
    print(prompt)
    print()

    complete = False
    previous_messages = []

    while not complete:
        executor_result = executor.loop(
            prompt=prompt,
            previous_messages=previous_messages,
            callback=callback,
            output_format=ExecutionReport
        )
        
        previous_messages.extend(executor_result.new_messages)

        print('cost:', executor_result.cost.total_cost)
        total_cost = total_cost + executor_result.cost

        print('complete:', executor_result.last_message.complete)
        complete = executor_result.last_message.complete
    
        iteration_updated_files = find_updated_files(executor_result.new_messages)
        all_changed_files = all_changed_files | iteration_updated_files

    print()

implementing step #9: Add email reminder management command...
1) Create myapp/management/commands/send_deadline_reminders.py.
2) Implement command logic:
- Determine current timezone-aware now() and the next 24 hours window.
- Query tasks where is_completed=False and due_date is between now and now+24h OR overdue within past day (optional).
- For each task, compute recipients: owner, shared_with users, and all team members (via TeamMembership.user). Use set to de-duplicate and skip inactive users.
- Send an email per recipient summarizing tasks due soon (group tasks per recipient to send one email per recipient). Use console backend as configured.
- Command output should print a summary of recipients and counts to stdout for testing.
3) Document in README (optional) the command usage: python manage.py send_deadline_reminders.
prompt:
Project overview: <OVERVIEW>Implement a multi-user to-do list with task creation/management, prioritization, deadline reminders, task sharing, comments, 

-> Response received


-> Response received


-> Response received


-> Response received
